In [1]:
pkg_ver = lambda pkg: "{:<20}{:}".format(pkg.__name__,pkg.__version__)

# ROOT
import uproot
print(pkg_ver(uproot))
import ROOT

# Machine Learning
import sklearn
print(pkg_ver(sklearn))
import torch
print(pkg_ver(torch))

# Data science
import scipy
print(pkg_ver(scipy))
import numpy
print(pkg_ver(numpy))
import pandas
print(pkg_ver(pandas))

# Visualizations
import matplotlib
print(pkg_ver(matplotlib))
import matplotlib.pyplot as plt

import tqdm
print(pkg_ver(tqdm))
import urllib.request
print(pkg_ver(urllib.request))

uproot              4.3.5
Welcome to JupyROOT 6.26/10
sklearn             1.2.0
torch               1.13.1
scipy               1.10.1
numpy               1.24.2
pandas              1.5.3
matplotlib          3.6.2
tqdm                4.62.3
urllib.request      3.10


In [2]:
%jsroot

In [3]:
class spectrum:
    
    def __init__(self, TH1D_, iTED_, Crystal_, Configuration_, Window_):  

        self.__TH1D = TH1D_
        self.__iTED = iTED_
        self.__Crystal = Crystal_
        self.__Configuration = Configuration_
        self.__Window = Window_
        
    def __call__(self, ch):
        return numpy.polyval(self.__Calibration[::-1],ch)
    
    def TH1D(self):
        return self.__TH1D
    
    def iTED(self):
        return self.__iTED
    
    def Crystal(self):
        return self.__Crystal
    
    def Configuration(self):
        return self.__Configuration
    
    def Window(self):
        return self.__Window
    
    def __repr__(self):
        return "iTED: {},Crystal: {},Configuration: {},Window: {}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window
        )
    
    def __str__(self):
        return "{}.{}.{}.{}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window
        )

In [4]:
ited = ["A","B","C","D"]
crystal = [0,1,2,3,4]

spectra = pandas.DataFrame(index = crystal, columns = ited)

# Access using spectra[iTED,Crystal][Configuration,Window]

spectra

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [5]:
iTEDA  = ROOT.TFile.Open("../../data/2023-03-02/888/CW100ns/Eu152-iTEDA-middle_D.2023_03_02_T.16_34_30_C.itedABCD_lab_2023.02.22_4.0v_888_300s.root","READ")
iTEDB  = ROOT.TFile.Open("../../data/2023-03-02/888/CW100ns/Eu152-iTEDB-middle_D.2023_03_02_T.17_10_31_C.itedABCD_lab_2023.02.22_4.0v_888_300s.root","READ")
iTEDC  = ROOT.TFile.Open("../../data/2023-03-02/888/CW100ns/Eu152-iTEDC-middle_D.2023_03_02_T.17_17_42_C.itedABCD_lab_2023.02.22_4.0v_888_300s.root","READ")
iTEDD  = ROOT.TFile.Open("../../data/2023-03-02/888/CW100ns/Eu152-iTEDD-middle_D.2023_03_02_T.17_23_40_C.itedABCD_lab_2023.02.22_4.0v_888_300s.root","READ")

In [6]:
for iTED in[0,1,2,3]:
    for Crystal in [0,1,2,3,4]:
            
        spectra[
            ["A","B","C","D"][iTED]
        ][
            Crystal
        ] = spectrum(
            [iTEDA,iTEDB,iTEDC,iTEDD][iTED].Get(
                "{}_{}_amplitude_spectra;1".format(
                    "SCATTERER" if Crystal==0 else "ABSORBER",
                    ["A","B","C","D"][iTED] if Crystal==0 else "{}_{}".format(["A","B","C","D"][iTED],Crystal)
                )
            ), 
            ["A","B","C","D"][iTED], 
            Crystal, 
            888, 
            100
        )

In [7]:
spectra.dropna(inplace=True)
spectra.applymap(str)

,A,B,C,D
0,A.0.888.100,B.0.888.100,C.0.888.100,D.0.888.100
1,A.1.888.100,B.1.888.100,C.1.888.100,D.1.888.100
2,A.2.888.100,B.2.888.100,C.2.888.100,D.2.888.100
3,A.3.888.100,B.3.888.100,C.3.888.100,D.3.888.100
4,A.4.888.100,B.4.888.100,C.4.888.100,D.4.888.100


In [8]:
def TH1D_draw(cell):
    
    TH1D = cell.TH1D()
    
    canvas = ROOT.TCanvas()
    canvas.cd()
    
    TH1D.SetTitle(repr(cell))
    TH1D.SetStats(False)
    
    latex = ROOT.TLatex()
    latex.SetNDC()
    latex.SetTextSize(0.03)
    
    TH1D.Draw("pe")
    
    l1,l2,l3 = get_resolution(cell)
    
    l4 = uproot.open(f"../../data/nTOF_March2022/{cell.Configuration()}/CW100ns/Resolutions_Cs137_CenterScatter_iTED{cell.iTED()}_8.8.{str(cell.Configuration())[2:]}_100ns.root:grResolEnergy;1").values()[1][cell.Crystal()]
    
    latex.DrawText(0.7, 0.8, "R_ch: {:.2f}%".format(l1))
    
    if l4*0.9 < l2 < l4*1.1:
        latex.DrawText(0.7, 0.75, "R_E: {:.2f}%".format(l2))
    else:
        latex.DrawText(0.7, 0.75, "->R_E: {:.2f}%".format(l2))
        
    if 662*(1-l2/100) < l3 < 662*(1+l2/100):
        latex.DrawText(0.7, 0.7, "E: {:.0f}keV".format(l3))
    else:
        latex.DrawText(0.7, 0.7, "->E: {:.0f}keV".format(l3))
        
    latex.DrawText(0.7, 0.65, "R_E(old): {:.2f}%".format(l4))
    latex.DrawText(0.7, 0.6, "Change: {:.2f}%".format(l2-l4))
            
    return canvas

In [9]:
def lc_read_csv(url):
    livechart = "https://nds.iaea.org/relnsd/v1/data?"
    url = livechart + url

    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    
    df = pandas.read_csv(urllib.request.urlopen(req), usecols=["energy", "intensity"])
    
    df = df[df['intensity'].str.strip().astype(bool)]

    df.sort_values(by=['intensity', 'energy'],ascending=[False, True], inplace=True, ignore_index=True)
    
    df = df.astype(numpy.half)
        
    return df

df = lc_read_csv("fields=decay_rads&nuclides=152eu&rad_types=g")

In [10]:
df = df[(df.energy>100) & (df.intensity>7)]

In [11]:
df.sort_values(by=['energy', 'intensity'],ascending=[True, False], inplace=True, ignore_index=True)

In [12]:
df

,energy,intensity
0,121.8125,28.531250
1,244.7500,7.550781
2,344.2500,26.593750
3,779.0000,12.929688
4,841.5000,14.203125
5,963.5000,11.601562
6,964.0000,14.507812
7,1086.0000,10.109375
8,1112.0000,13.671875
9,1408.0000,20.875000
